In [2]:
# 数据预处理
# 创建一个人工数据集，并存储在csv(逗号分隔值)文件中
import torch
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 创建一个名为 'data' 的目录，如果该目录已经存在，则不会报错
os.makedirs(os.path.join('..', 'data'), exist_ok=True)

# 定义数据文件的路径，文件名为 'house_tiny.csv'
data_file = os.path.join('..', 'data', 'house_tiny.csv')

# 打开文件进行写操作，如果文件不存在则创建文件
with open(data_file, 'w') as f:
    # 写入列名
    f.write('NumRooms,Alley,Price\n')  # 列名(房间数，巷子类型，价格)
    # 写入第一行数据，表示一个数据样本
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本(本行：房间数未知，巷子类型为“Pave”，价格为127500)
    # 写入第二行数据
    f.write('2,NA,106000\n')
    # 写入第三行数据
    f.write('4,Brick,178100\n')  # Brick 表示“砖砌”
    # 写入第四行数据
    f.write('NA,NA,140000\n')

In [4]:
# 读取数据集, 一般用的是pandas的read_csv函数
data = pd.read_csv(data_file)
data  # jupyter中直接输出data，会显示表格形式的数据

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,Brick,178100
3,NaN,NaN,140000


In [5]:
# 为了处理缺失的数据，典型的方法包括插值和删除，这里我们将考虑插值
# 假设 data 是之前读取的 DataFrame
data = pd.read_csv(data_file)

# inputs 包含前两列（NumRooms 和 Alley），outputs 包含第三列（Price）
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]

# 只对数值列进行插值
# 选择 inputs 中的数值列
numeric_inputs = inputs.select_dtypes(include=[np.number])

# 使用数值列的均值(此处为2.0 + 4.0 >> 1 = 3.0)填充缺失值
inputs[numeric_inputs.columns] = numeric_inputs.fillna(numeric_inputs.mean())

# 打印处理后的 inputs, outputs
print(inputs)
print(outputs)

   NumRooms  Alley
0       3.0   Pave
1       2.0    NaN
2       4.0  Brick
3       3.0    NaN
0    127500
1    106000
2    178100
3    140000
Name: Price, dtype: int64


In [6]:
# 对于 inputs 中的类别值或离散值，我们将 “NaN” 视为一个类别
# get_dummies 将 “NaN” 视为有效的类别值，并为其创建指示特征
inputs = pd.get_dummies(inputs, dummy_na=True)  # dummy_na=True 将缺失值也当作合法的特征值并为其创建指示特征
inputs = inputs.astype(int)  # 转换数据类型为整数
print(inputs)

   NumRooms  Alley_Brick  Alley_Pave  Alley_nan
0         3            0           1          0
1         2            0           0          1
2         4            1           0          0
3         3            0           0          1


In [7]:
# 现在 inputs 和 outputs 中的所有条目都是数值类型，它们可以转换为张量格式
# 如有需要，将 outputs 也转换为数值(整数)类型
# outputs = outputs.astype(int)

# 转换为张量格式
x, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
x, y

(tensor([[3, 0, 1, 0],
         [2, 0, 0, 1],
         [4, 1, 0, 0],
         [3, 0, 0, 1]], dtype=torch.int32),
 tensor([127500, 106000, 178100, 140000]))